In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
path = r'C:\Users\User\.cache\kagglehub\datasets\nih-chest-xrays\sample\versions\4'

# Explore the raw dataset

In [3]:
df = pd.read_csv(path+'\sample_labels.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [4]:
df.shape

(5606, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [6]:
#checking for duplicates
df.duplicated().sum()

0

In [7]:
#checking for missing data
df.isna().sum()

Image Index                    0
Finding Labels                 0
Follow-up #                    0
Patient ID                     0
Patient Age                    0
Patient Gender                 0
View Position                  0
OriginalImageWidth             0
OriginalImageHeight            0
OriginalImagePixelSpacing_x    0
OriginalImagePixelSpacing_y    0
dtype: int64

In [8]:
#checking the balance level of data
df['Finding Labels'].value_counts(normalize = True)

Finding Labels
No Finding                                                0.542990
Infiltration                                              0.089725
Effusion                                                  0.036211
Atelectasis                                               0.034249
Nodule                                                    0.025687
                                                            ...   
Atelectasis|Edema|Effusion|Infiltration|Pneumonia         0.000178
Atelectasis|Consolidation|Edema|Infiltration|Pneumonia    0.000178
Atelectasis|Effusion|Hernia                               0.000178
Atelectasis|Hernia|Pneumothorax                           0.000178
Cardiomegaly|Effusion|Emphysema                           0.000178
Name: proportion, Length: 244, dtype: float64

In [9]:
labels = ['Hernia', 
          'Pneumonia', 
          'Fibrosis', 
          'Edema', 
          'Emphysema', 
          'Cardiomegaly', 
          'Pleural_Thickening', 
          'Consolidation', 
          'Pneumothorax', 
          'Mass', 
          'Nodule', 
          'Atelectasis',
          'Effusion', 
          'Infiltration']

In [10]:
#drop rows with 'No Finding'
filtered_df = df[df['Finding Labels']!= 'No Finding'] 
filtered_df.shape

(2562, 11)

In [11]:
filtered_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168
5,00000040_003.png,Consolidation|Mass,3,40,068Y,M,PA,2500,2048,0.168,0.168


In [12]:
#drop unwanted columns
filtered_df = filtered_df.drop(['View Position', 'OriginalImageWidth', 'OriginalImageHeight', 'OriginalImagePixelSpacing_x', 'OriginalImagePixelSpacing_y'], axis =1 )
filtered_df = filtered_df.reset_index()
filtered_df.head()

,index,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender
0,0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M
1,1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M
2,3,00000030_001.png,Atelectasis,1,30,079Y,M
3,4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F
4,5,00000040_003.png,Consolidation|Mass,3,40,068Y,M


In [13]:
my_data= pd.DataFrame( columns= labels)
my_data.head()

,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration


In [14]:
my_data= pd.concat([filtered_df, my_data], axis=1)
my_data['Finding Labels']=my_data['Finding Labels'].astype(str)
my_data['Finding Labels']=my_data['Finding Labels'].str.split('|')
my_data.head()

,index,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,Hernia,Pneumonia,Fibrosis,...,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
0,0,00000013_005.png,"[Emphysema, Infiltration, Pleural_Thickening, ...",5,13,060Y,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,00000013_026.png,"[Cardiomegaly, Emphysema]",26,13,057Y,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,00000030_001.png,[Atelectasis],1,30,079Y,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,00000032_001.png,"[Cardiomegaly, Edema, Effusion]",1,32,055Y,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,00000040_003.png,"[Consolidation, Mass]",3,40,068Y,M,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for i in range(len(my_data)):
    temp=my_data.loc[i, 'Finding Labels']
    for j in labels:
        if j in temp:
            my_data.loc[i,j]=1
        else:
            my_data.loc[i,j]=0

my_data = my_data.drop(['index', 'Finding Labels'] , axis =1)
my_data.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
0,00000013_005.png,5,13,060Y,M,0,0,0,0,1,0,1,0,1,0,0,0,0,1
1,00000013_026.png,26,13,057Y,M,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,00000030_001.png,1,30,079Y,M,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,00000032_001.png,1,32,055Y,F,0,0,0,1,0,1,0,0,0,0,0,0,1,0
4,00000040_003.png,3,40,068Y,M,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [17]:
#save my_data as csv
my_data.to_csv('my_data.csv', index =True)